# IBM Quantum Hardware Validation Demo

## Entanglement Distillation on Real Quantum Hardware

This notebook demonstrates:
1. Hardware-compatible BBPSSW distillation circuits
2. Backend selection and calibration monitoring
3. Execution on IBM Quantum hardware
4. Fidelity estimation from measurements
5. Comparison with simulation

**IMPORTANT**: This is a hardware validation prototype. IBM Quantum does NOT support real quantum networking.

## Setup

In [ ]:
# Import required modules
from ibm_hardware import IBMQuantumHardwareValidator, plot_fidelity_comparison, save_validation_report
import matplotlib.pyplot as plt
import numpy as np
import os

# Configure matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## Configuration

**Get your API token from**: https://quantum.ibm.com/account

In [ ]:
# Try to import from config file
try:
    from ibm_config import IBM_API_TOKEN, DEFAULT_SHOTS, USE_REAL_HARDWARE, MIN_QUBITS
    print("✓ Loaded configuration from ibm_config.py")
    SHOTS = DEFAULT_SHOTS
except ImportError:
    # Fall back to environment variable or manual entry
    IBM_API_TOKEN = os.environ.get('IBM_QUANTUM_TOKEN', 'your_token_here')
    SHOTS = 4096
    USE_REAL_HARDWARE = False
    MIN_QUBITS = 5
    
    if IBM_API_TOKEN == 'your_token_here':
        print("⚠️  Please set your IBM Quantum API token:")
        print("   1. Create ibm_config.py from ibm_config_template.py")
        print("   2. OR set IBM_API_TOKEN variable in this cell")
        print("   3. OR set environment variable: IBM_QUANTUM_TOKEN")
        print("\nGet your token from: https://quantum.ibm.com/account")

## Step 1: Initialize Connection

In [ ]:
# Initialize validator
validator = IBMQuantumHardwareValidator(
    api_token=IBM_API_TOKEN,
    hub="ibm-q",
    group="open",
    project="main"
)

print("✓ Connected to IBM Quantum")

## Step 2: Select Backend

In [ ]:
# Select best available backend
backend_info = validator.select_best_backend(
    min_qubits=MIN_QUBITS,
    simulator=not USE_REAL_HARDWARE,
    verbose=True
)

print(f"\n✓ Backend selected: {backend_info['name']}")

## Step 3: Create Hardware-Compatible Circuit

### BBPSSW Distillation Protocol

Uses 2 Bell pairs (4 qubits):
- **Pair 0** (q0, q1): Data pair (target)
- **Pair 1** (q2, q3): Ancilla pair

**Protocol**:
1. Prepare Bell pairs: H-CX on each pair
2. Bilateral CNOT: q0→q2, q1→q3
3. Measure ancilla qubits (q2, q3)
4. Post-select on ancilla = |00⟩

In [ ]:
# Create hardware-compatible BBPSSW circuit
circuit, metadata = validator.create_hardware_bbpssw_circuit(
    prepare_bell_pairs=True,
    measure_output=True
)

print("Circuit Metadata:")
print(f"  Protocol: {metadata['protocol']}")
print(f"  Qubits: {metadata['num_qubits']}")
print(f"  Depth: {metadata['depth']}")
print(f"  Target qubits: {metadata['target_qubits']}")
print(f"  Ancilla qubits: {metadata['ancilla_qubits']}")
print(f"  Success condition: {metadata['success_condition']}")

print("\nCircuit Diagram:")
print(circuit.draw(output='text', fold=-1))

## Step 4: Create Fidelity Measurement Circuits

To estimate Bell state fidelity, we measure in multiple bases:
- **ZZ**: Computational basis (Z⊗Z)
- **XX**: X-basis (apply H before measurement)
- **YY**: Y-basis (apply S†H before measurement)

Fidelity formula: **F = (1 + ⟨ZZ⟩ + ⟨XX⟩ + ⟨YY⟩) / 4**

In [ ]:
# Create fidelity measurement circuits
fidelity_circuits = validator.create_fidelity_measurement_circuits(
    prepare_bell_pairs=True
)

print(f"Created {len(fidelity_circuits)} measurement circuits:\n")

for name, circ in fidelity_circuits.items():
    print(f"\n{'='*60}")
    print(f"{name} Measurement Circuit")
    print(f"{'='*60}")
    print(f"Qubits: {circ.num_qubits}, Depth: {circ.depth()}")
    print(circ.draw(output='text', fold=-1))

## Step 5: Execute on Hardware

**Note**: If `USE_REAL_HARDWARE = False`, this uses AerSimulator (local simulation).

In [ ]:
if USE_REAL_HARDWARE:
    print("⚠️  WARNING: Using real quantum hardware")
    print("This will use queue time and may take several minutes.\n")

# Execute circuits
print(f"Executing {len(fidelity_circuits)} circuits with {SHOTS} shots each...\n")

hardware_results = validator.execute_multiple_circuits(
    fidelity_circuits,
    shots=SHOTS,
    optimization_level=3,
    verbose=True
)

print("\n✓ All circuits executed successfully")

## Step 6: Analyze Results

### Measurement Statistics

In [ ]:
# Display measurement counts
for name, result in hardware_results.items():
    print(f"\n{name} Measurement:")
    print(f"  Backend: {result['backend']}")
    print(f"  Shots: {result['shots']}")
    print(f"  Transpiled depth: {result['transpiled_depth']}")
    print(f"  Execution time: {result['execution_time']:.2f}s")
    
    counts = result['counts']
    print(f"\n  Top 5 outcomes:")
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]
    for bitstring, count in sorted_counts:
        prob = count / result['shots']
        print(f"    {bitstring}: {count} ({prob:.2%})")

### Visualize Measurement Distributions

In [ ]:
# Plot measurement distributions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (name, result) in enumerate(hardware_results.items()):
    counts = result['counts']
    
    # Get top 8 outcomes
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:8]
    labels = [x[0] for x in sorted_counts]
    values = [x[1] for x in sorted_counts]
    
    # Plot
    axes[idx].bar(range(len(labels)), values, color='steelblue', alpha=0.7, edgecolor='black')
    axes[idx].set_xlabel('Measurement Outcome', fontsize=12)
    axes[idx].set_ylabel('Counts', fontsize=12)
    axes[idx].set_title(f'{name} Basis Measurements', fontsize=14, fontweight='bold')
    axes[idx].set_xticks(range(len(labels)))
    axes[idx].set_xticklabels(labels, rotation=45, ha='right')
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Step 7: Estimate Fidelity

In [ ]:
# Estimate Bell state fidelity
fidelity_estimate = validator.estimate_bell_state_fidelity(
    hardware_results,
    target_state='phi_plus'
)

print("\n" + "="*60)
print("FIDELITY ESTIMATION RESULTS")
print("="*60)

print(f"\nTarget state: |Φ+⟩ = (|00⟩ + |11⟩)/√2")
print(f"\nEstimated fidelity: {fidelity_estimate['fidelity']:.4f} ± {fidelity_estimate['error']:.4f}")
print(f"95% Confidence interval: [{fidelity_estimate['fidelity_lower']:.4f}, {fidelity_estimate['fidelity_upper']:.4f}]")

print(f"\nExpectation values:")
print(f"  ⟨ZZ⟩ = {fidelity_estimate['zz_expectation']:.4f}")
print(f"  ⟨XX⟩ = {fidelity_estimate['xx_expectation']:.4f}")
print(f"  ⟨YY⟩ = {fidelity_estimate['yy_expectation']:.4f}")

print(f"\nComputational basis probabilities:")
probs = fidelity_estimate['probabilities']
print(f"  P(|00⟩) = {probs['p_00']:.4f}")
print(f"  P(|11⟩) = {probs['p_11']:.4f}")
print(f"  P(|01⟩) = {probs['p_01']:.4f}")
print(f"  P(|10⟩) = {probs['p_10']:.4f}")

print(f"\nTotal shots: {fidelity_estimate['total_shots']}")

# Interpretation
if fidelity_estimate['fidelity'] > 0.5:
    print("\n✓ Fidelity > 0.5: Entanglement detected!")
else:
    print("\n✗ Fidelity ≤ 0.5: No entanglement (or measurement error)")

## Step 8: Post-Selection Analysis

In [ ]:
# Apply post-selection (keep only shots where ancilla = |00⟩)
zz_counts = hardware_results['ZZ']['counts']
post_selected, success_prob = validator.apply_post_selection(
    zz_counts,
    success_condition='ancilla_00'
)

print("\n" + "="*60)
print("POST-SELECTION ANALYSIS")
print("="*60)

print(f"\nSuccess condition: Ancilla qubits = |00⟩")
print(f"\nSuccess probability: {success_prob:.2%}")
print(f"Successful shots: {sum(post_selected.values())}")
print(f"Total shots: {sum(zz_counts.values())}")
print(f"Rejected shots: {sum(zz_counts.values()) - sum(post_selected.values())}")

print(f"\nPost-selected outcomes:")
sorted_post = sorted(post_selected.items(), key=lambda x: x[1], reverse=True)[:5]
for bitstring, count in sorted_post:
    prob = count / sum(post_selected.values())
    print(f"  {bitstring}: {count} ({prob:.2%})")

## Step 9: Compare with Simulation

In [ ]:
# Run simulation with backend noise model
print("Running noisy simulation for comparison...\n")

from qiskit_aer import AerSimulator
from qiskit import transpile as basic_transpile

# Get noise model from backend
noise_model = validator.get_noise_model_from_backend()

if noise_model:
    print("✓ Using backend noise model")
    simulator = AerSimulator(noise_model=noise_model)
else:
    print("✓ Using ideal simulator (no noise model available)")
    simulator = AerSimulator()

# Run simulation
sim_results = {}
for name, circuit in fidelity_circuits.items():
    transpiled = basic_transpile(circuit, simulator)
    job = simulator.run(transpiled, shots=SHOTS)
    result = job.result()
    sim_results[name] = {
        'counts': result.get_counts(),
        'shots': SHOTS
    }

# Estimate fidelity from simulation
sim_fidelity = validator.estimate_bell_state_fidelity(
    sim_results,
    target_state='phi_plus'
)

print("\n" + "="*60)
print("HARDWARE vs SIMULATION COMPARISON")
print("="*60)

print(f"\nHardware fidelity:    {fidelity_estimate['fidelity']:.4f} ± {fidelity_estimate['error']:.4f}")
print(f"Simulation fidelity:  {sim_fidelity['fidelity']:.4f} ± {sim_fidelity['error']:.4f}")
print(f"\nDifference: {abs(fidelity_estimate['fidelity'] - sim_fidelity['fidelity']):.4f}")

if fidelity_estimate['fidelity'] > sim_fidelity['fidelity']:
    print("→ Hardware performed BETTER than simulation")
else:
    print("→ Simulation performed better (expected for noisy hardware)")

## Step 10: Visualize Comparison

In [ ]:
# Plot fidelity comparison
plot_fidelity_comparison(
    hardware_fidelity=fidelity_estimate['fidelity'],
    hardware_error=fidelity_estimate['error'],
    simulation_fidelity=sim_fidelity['fidelity'],
    simulation_error=sim_fidelity['error'],
    save_path="fidelity_comparison.png"
)

## Step 11: Generate Validation Report

In [ ]:
# Create comprehensive report
report = {
    'timestamp': hardware_results['ZZ']['timestamp'],
    'backend': backend_info,
    'circuits': {
        name: {
            'depth': circ.depth(),
            'num_qubits': circ.num_qubits
        }
        for name, circ in fidelity_circuits.items()
    },
    'hardware_results': {
        'fidelity': fidelity_estimate,
        'post_selection': {
            'success_probability': success_prob,
            'successful_shots': sum(post_selected.values()),
            'total_shots': sum(zz_counts.values())
        }
    },
    'simulation_results': {
        'fidelity': sim_fidelity,
        'noise_model': 'backend_calibration' if noise_model else 'ideal'
    },
    'comparison': {
        'fidelity_difference': abs(fidelity_estimate['fidelity'] - sim_fidelity['fidelity']),
        'hardware_better': fidelity_estimate['fidelity'] > sim_fidelity['fidelity']
    }
}

# Save report
save_validation_report(report, filename="ibm_validation_report.json")

print("\n✓ Validation report saved to: ibm_validation_report.json")

## Summary

### Key Findings

In [ ]:
print("\n" + "="*70)
print("VALIDATION SUMMARY")
print("="*70)

print(f"\nBackend: {backend_info['name']}")
if not backend_info.get('simulator', False):
    print(f"  Qubits: {backend_info['num_qubits']}")
    print(f"  Avg CX error: {backend_info['avg_cx_error']:.4f}")
    print(f"  Avg T1: {backend_info['avg_t1_us']:.1f} μs")
    print(f"  Avg T2: {backend_info['avg_t2_us']:.1f} μs")

print(f"\nCircuit:")
print(f"  Protocol: BBPSSW")
print(f"  Qubits: 4 (2 Bell pairs)")
print(f"  Depth: {metadata['depth']}")

print(f"\nExecution:")
print(f"  Shots per circuit: {SHOTS}")
print(f"  Total circuits: {len(fidelity_circuits)}")
print(f"  Total shots: {SHOTS * len(fidelity_circuits)}")

print(f"\nResults:")
print(f"  Hardware fidelity: {fidelity_estimate['fidelity']:.4f} ± {fidelity_estimate['error']:.4f}")
print(f"  Simulation fidelity: {sim_fidelity['fidelity']:.4f} ± {sim_fidelity['error']:.4f}")
print(f"  Post-selection success: {success_prob:.2%}")

print(f"\nInterpretation:")
if fidelity_estimate['fidelity'] > 0.5:
    print(f"  ✓ Entanglement verified (F > 0.5)")
else:
    print(f"  ✗ No entanglement detected (F ≤ 0.5)")

if fidelity_estimate['fidelity'] > 0.7:
    print(f"  ✓ High-quality entanglement (F > 0.7)")
elif fidelity_estimate['fidelity'] > 0.5:
    print(f"  ⚠ Low-quality entanglement (0.5 < F < 0.7)")

print(f"\nFiles generated:")
print(f"  - ibm_validation_report.json")
print(f"  - fidelity_comparison.png")

print("\n" + "="*70)
print("✓ VALIDATION COMPLETE")
print("="*70)

## Next Steps

1. **Try different backends**: Compare results across multiple IBM devices
2. **Vary shot counts**: Test how fidelity estimates converge with more shots
3. **Test with real hardware**: Set `USE_REAL_HARDWARE = True` to run on actual quantum computers
4. **Optimize circuits**: Experiment with different transpiler optimization levels
5. **Compare protocols**: Implement DEJMPS and compare with BBPSSW

## Resources

- **IBM Quantum**: https://quantum.ibm.com/
- **Qiskit Documentation**: https://docs.quantum.ibm.com/
- **BBPSSW Paper**: Bennett et al. (1996) - "Purification of noisy entanglement"
- **Project README**: See `IBM_HARDWARE_README.md` for detailed documentation